In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/tf/notebooks/src


## Intialization

In [3]:
try:
    import pandas
except:
    !pip install pandas iterative-stratification nlpaug==0.0.20 tqdm click tensorflow_probability==0.11.1 tf2_resnets tensorflow_addons==0.11.1 image-classifiers==0.2.2

### Imports

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from collections import Counter

In [5]:
from utils.metric import lwlrap

In [6]:
from params import *
from train import get_model
from params import TEST_MELS_PATH, TRAIN_MELS_PATH

In [7]:
# CP_FOLDER = LOG_PATH + "2021-02-04/9/"
# CP_FOLDER = LOG_PATH + "2021-02-04/12/"
# CP_FOLDER = "../data/densenet121_pretrained/"  # 0.897

In [8]:
CP_FOLDERS = [
#     LOG_PATH + "2021-02-04/9/",
    "../logs/2021-02-13/13/"
#     LOG_PATH + "2021-02-04/12/",
]

# CP_FOLDERS = [
#     "../logs/2021-02-13/0/",
#     "../logs/2021-02-13/1/",
#     "../logs/2021-02-13/2/",
#     "../logs/2021-02-13/3/",
#     "../logs/2021-02-13/4/",
#     "../logs/2021-02-13/5/",
# ]

## Test

In [9]:
sub = pd.read_csv(DATA_PATH + "sample_submission.csv")

In [10]:
ref = pd.read_csv(DATA_PATH + "sub930_top5.csv")

In [11]:
preds = []

for cp in CP_FOLDERS:
    for fold_idx in range(5):
        try:
            pred = np.load(cp + f"preds/pred_agg_{fold_idx}.npy")
            preds.append(pred)
            
            break
        except:
            pass
    
preds = np.mean(preds, 0)

In [12]:
score = lwlrap(ref[ref.columns[1:]].values, preds)[-1]
print(f'Ref score : {score:.4f}')

Ref score : 0.8338


In [39]:
sub[sub.columns[1:]] = preds

In [40]:
sub.to_csv(OUT_DIR + F"submission_{score:.4f}.csv", index=False)

print(f'Saved submission to {OUT_DIR + F"submission_{score:.4f}.csv"}')

Saved submission to ../output/submission_0.9447.csv


## Train